# 실무예제 3-2

## 다음은 2013년 전라남도 유망중소기업 지정업체 명단의 일부이다. 시군 속성과 지정구분 속성 간의 연관성 여부를 카이제곱 검정 방법에 의해 판단해 보시오. (단, 유의 수준은 0.05이다)

### 데이터 파일 : ch3-2(유망중소기업현황).csv

### 원본 투플수 : 386개

In [1]:
# ch3-2.py
import pandas as pd
import numpy as np
import cx_Oracle      # Oracle DB 연동을 위한 cx_Oracle 패키지 임포트

# 데이터로드 (ch2-2.csv : 데이터 원본 파일)
# encoding : 윈도우즈 환경에서의 한글 처리
# engine : python 3.6에서 한글이 포함된 파일이름 사용
rawData_org = pd.read_csv('.jupyter/ch3-2(유망중소기업현황).csv', encoding='CP949', engine='python')

In [2]:
rawData_org

,연번,시군,지정구분,기업명,대표자,소 재 지,주생산품,전화번호(061),비고\r\n(지정번호)
0,1,목포시,기술유망,브로드컴㈜,이봉헌,목포시 석현동 1175(벤처지원 202),정보통신,284-0017,11-1
1,2,목포시,기술유망,㈜케이에스,김시오,목포시 연산동1236-3,용접철망외,1588-4118,11-2
2,3,목포시,기술유망,삼진물산(주),김관석,목포시 연산동 1239-1,참치통조림,270-6113,11-3
3,4,목포시,기술유망,㈜혜성,전재두,목포시 연산동1237-3,가드레일외,1588-2811,11-4
4,5,목포시,기술유망,(유)한국메이드,이승룡,목포시 연산동 1238-4,선박블록,278-4411,11-5
...,...,...,...,...,...,...,...,...,...
381,382,완도군,지역특화,㈜세영산업,김주효,완도군 완도읍가용리,찐톳.건미역,555-0294,11-276
382,383,완도군,기술유망,동주씨테크㈜,이용철,완도군 완도읍 가용리 1086-6,개량부자 전복양식자재,554-6690,12-63
383,384,진도군,수출유망,대대로영농조합법인,김애란,진도군 군내면 명량대첩로 288-23,홍주,542-3399,11-290
384,385,진도군,지역특화,진도정미소영농조합법인,이운갑,진도군 의신면 돈지리 144,"흑미, 기타유색미",544-3459,12-64


In [3]:
# 원본 데이터파일에서 '시군'과 '지정구분' 속성만 추출
rawData = rawData_org.loc[:,['시군', '지정구분']]

In [4]:
rawData

,시군,지정구분
0,목포시,기술유망
1,목포시,기술유망
2,목포시,기술유망
3,목포시,기술유망
4,목포시,기술유망
...,...,...
381,완도군,지역특화
382,완도군,기술유망
383,진도군,수출유망
384,진도군,지역특화


In [10]:
# Oracle DB 연결
# 접속정보(connection string) : ID/PASS@CONNECTION_ALIAS
# CONNECTION_ALIAS : Oracle TNSNAMES.ORA 파일에 있는 접속정보 별칭(ALIAS)
conn_ora = cx_Oracle.connect("NOH/flqhs4747@XE")

# DB 커서(Cursor) 선언
cur = conn_ora.cursor()

# 사용할 Oracle 소스 테이블명 지정
src_table = "d_base3_2" # 이것과

# 데이터프레임(rawData)에 저장된 데이터를 Oracle 테이블(d_base3_2)에 입력하기 위한 로직
# d_base3_2 테이블 존재하는지 체크하는 함수
def table_exists(name=None, con=None):
    sql = "select table_name from user_tables where table_name='MYTABLE'".replace('MYTABLE', name.upper())
    df = pd.read_sql(sql, con)

    # 테이블이 존재하면 True, 그렇지 않으면 False 반환
    exists = True if len(df) > 0 else False
    return exists

# 테이블(d_base3_2) 생성 (테이블이 이미 존재한다면 TRUNCATE TABLE)
if table_exists(src_table, conn_ora):
    cur.execute("TRUNCATE TABLE " + src_table)
else:
    cur.execute("create table " + src_table + " ( \
               시군 varchar2(10), \
               지정구분 varchar2(20))") # 이것만 바꿔주며 된다?

# Sequence 구조를 Dictionary 구조((element, value))로 변환하는 함수
# 예: ("Matt", 1) -> {'1':'Matt', '2':1}
# INSERT INTO ... VALUES (:1, :2, ...) 에서 바인드 변수값을 주기위해 Dictionary item 구조 사용
def convertSequenceToDict(list):
    dict = {}
    argList = range(1, len(list) + 1)
    for k, v in zip(argList, list):
        dict[str(k)] = v
    return dict

# 데이터프레임에 저장된 데이터를 Oracle 테이블로 입력(insert)
cols = [k for k in rawData.dtypes.index]
colnames = ','.join(cols)
colpos = ', '.join([':' + str(i + 1) for i, f in enumerate(cols)])
insert_sql = 'INSERT INTO %s (%s) VALUES (%s)' % (src_table, colnames, colpos)

# INSERT INTO ... VALUES (:1, :2, ...)의 바인드 변수 값을 저장하는 Dictionary 구조 생성
data = [convertSequenceToDict(rec) for rec in rawData.values]

# 바인드 변수와 Dictionary 데이터구조를 활용하여 Bulk Insertion 구현
cur.executemany(insert_sql, data)
conn_ora.commit()

### 실무예제 2-2의 Oracle 연동 설명 참조
### Oracle sqlplus를 통해서 d_base3_2 테이블 생성 확인

In [10]:
# 관측도수/기대도수 분할표 생성 (1차원 배열 형식)
count_df = pd.read_sql("""""",con)#???

### pandas 데이터프레임에 카이제곱 검정을 위한 관측도수/기대도수 분할표 저장
### ‘시군’ 속성 ‘지정구분’ 속성에 대한 관측도수와 기대도수를 구하기 위해서 Oracle SQL을 활용
#### 관측도수 :  ‘시군’과 ‘지정구분’ 속성값의 조합에 대한 실제 출현횟수
#### 기대도수 : 전체 행의 개수와 각 속성의 카디널러티(cardinality, 서로 다른 속성값의 개수)를 구하여 산출
#### 실제 출현하지 않은 (시군+지정구분) 속성값도 포함시키기 위해서 외부조인(outer join)을 수행

In [11]:
# 관측도수/기대도수 분할표 출력
count_df

,시군,지정구분,관측도수,기대도수
0,목포시,기술유망,14,13.056995
1,나주시,수출유망,3,5.531088
2,보성군,수출유망,3,1.896373
3,보성군,지역특화,3,2.269430
4,해남군,지역특화,2,1.323834
5,무안군,수출유망,3,2.686528
6,무안군,기술유망,9,11.098446
7,영광군,지역특화,3,1.512953
8,완도군,기술유망,9,11.751295
9,신안군,기술유망,1,0.652850


In [12]:
# 2개 속성에 대한 자유도(degree of freedom) 갭 구하기
# A 속성에 대한 cardinality = a, B 속성에 대한 cardinality = b라 가정
# cardinality : 서로 다른 속성값의 개수
# 분할표 전체 행 갯수(a*b) 구하기


### ‘시군’ 속성의 카디널러티(a) = 22
### ‘지정구분’ 속성의 카디널러티(b) = 3
### 그래서, (시군+지정구분) 속성조합의 카디널러티(a\*b) = 22 * 3 = 66

In [13]:
tot_rows  # 변수확인

66

In [14]:
# A 속성의 cardianlity(a)와 B 속성의 cardinality(b) 구하기


In [15]:
count_df2  # 변수확인

,도수_시군,도수_지정구분
0,22,3


In [16]:
# cardinality 갭 [(a*b-1) - (a-1)*(b-1)] 구하기


### 속성조합의 자유도(degree of freedom) = (66-1) = 65
### 실제 자유도는 (22-1) * (3-1) = 42
### 위 두 값의 차이 : 23 -> chisquare() 함수의 세번째 패러미터로 사용

In [17]:
v_ddof  # 변수확인

0    23
dtype: int64

In [18]:
# obs_array : 관측도수를 저장하는 1차원 배열


### obs_array : chisquare() 함수의 첫번째 패러미터로 사용

In [19]:
obs_array  # 변수확인

0     14
1      3
2      3
3      3
4      2
5      3
6      9
7      3
8      9
9      1
10    23
11     1
12     1
13     8
14    29
15     6
16     3
17    10
18    37
19    23
20     4
21     4
22     3
23     6
24    14
25     1
26     3
27     1
28     2
29     4
      ..
36     6
37     1
38     5
39     2
40     4
41     6
42    12
43     6
44     3
45     5
46    22
47     5
48     1
49     2
50     1
51    16
52    15
53     1
54     1
55     7
56     2
57     1
58     3
59     6
60     0
61     0
62     0
63     0
64     0
65     0
Name: 관측도수, Length: 66, dtype: int64

In [20]:
# exp_array : 기대도수를 저장하는 1차원 배열


### exp_array : chisquare() 함수의 두번째 패러미터로 사용

In [21]:
exp_array  # 변수확인

0     13.056995
1      5.531088
2      1.896373
3      2.269430
4      1.323834
5      2.686528
6     11.098446
7      1.512953
8     11.751295
9      0.652850
10    22.849741
11     1.264249
12     3.782383
13     5.531088
14    24.808290
15     6.005181
16     7.186528
17     6.619171
18    32.642487
19    20.238342
20     1.323834
21     5.222798
22     2.844560
23     9.455959
24    11.098446
25     1.512953
26     4.898964
27     0.948187
28     3.002591
29     6.619171
        ...    
36     5.222798
37     1.264249
38     5.862694
39     4.569948
40     4.569948
41     5.373057
42     6.430052
43     5.875648
44     1.422280
45     3.160622
46    22.849741
47     3.917098
48     0.948187
49     3.917098
50     1.106218
51    22.196891
52    12.404145
53     0.316062
54     0.378238
55     7.901554
56     3.215026
57     0.948187
58     1.134715
59     3.404145
60     1.702073
61     0.189119
62     1.134715
63     0.158031
64     1.134715
65     1.305699
Name: 기대도수, Length: 66, 

In [23]:
# 카이제곱검정을 위한 scipy 패키지 중 stats 모듈 임포트
from scipy import stats

# stats.chisquare() : 카이제곱검정 함수


### chisquare() 함수 : 카이제곱(χ2) 통계량을 산출하는 함수
#### 첫번째 패러미터 : 1차원 배열 형태의 관측도수 리스트
#### 두번째 패러미터 : 1차원 배열 형태의 기대도수 리스트
#### 세번째 패러미터 : 자유도 차이 [(a\*b-1) - (a-1)\*(b-1)]

In [24]:
# stats.chisquare() 수행 후의 카이제곱 통계량과 p-value


statistic = 54.544, p-value=0.09287


### 카이제곱 통계량(statistic) : 54.544
### 유의확률(p-value) : 0.09287
### 유의확률이 유의수준 0.05로 설정한다면 귀무가설(두 속성은 연관성이 없다)이 채택됨. 결론적으로, 시군과 지정구분 사이에는 큰 연관성은 없는 것으로 판단할 수 있음.